In [0]:
%run ../Includes/Copy-Datasets

In [0]:
SELECT * FROM customers;

In [0]:
DESC customers;

In [0]:
SELECT customer_id, profile:first_name, profile:address:country FROM customers;


In [0]:
SELECT profile
FROM customers
LIMIT 1;

In [0]:
CREATE OR REPLACE TEMP VIEW parsed_customers AS
 SELECT customer_id, from_json(profile, schema_of_json('{"first_name":"Susana","last_name":"Gonnely","gender":"Female","address":{"street":"760 Express Court","city":"Obrenovac","country":"Serbia"}}')) AS profile_struct 
 FROM customers;

SELECT * FROM parsed_customers;

In [0]:
DESC parsed_customers;

In [0]:
SELECT customer_id, profile_struct.first_name, profile_struct.address.country FROM parsed_customers;

In [0]:
CREATE OR REPLACE TEMP VIEW customers_final AS
  SELECT customer_id, profile_struct.*
  FROM parsed_customers;
SELECT * FROM customers_final;

In [0]:
SELECT order_id, customer_id, books
FROM orders;

In [0]:
SELECT order_id, customer_id, explode(books) AS book
FROM orders;

In [0]:
SELECT customer_id,
      collect_set(order_id) AS order_set,
      collect_set(books.book_id) AS book_set
FROM orders
GROUP BY customer_id;

In [0]:
SELECT customer_id,
      collect_set(books.book_id) AS before_flatten,
      array_distinct(flatten(collect_set(books.book_id))) AS after_flatten
FROM orders
GROUP BY customer_id;

In [0]:
CREATE OR REPLACE VIEW orders_enriched AS
SELECT *
FROM (
  SELECT *, explode(books) AS book
  FROM orders) o
INNER JOIN books b
ON o.book.book_id = b.book_id;
SELECT * FROM orders_enriched;

In [0]:
CREATE OR REPLACE TEMP VIEW orders_updates AS
SELECT * FROM parquet.`${dataset.bookstore}/orders-new`;

SELECT * FROM orders
UNION
SELECT * FROM orders_updates

In [0]:
SELECT * FROM orders
INTERSECT
SELECT * FROM orders_updates

In [0]:
SELECT * FROM orders
MINUS
SELECT * FROM orders_updates

In [0]:
CREATE OR REPLACE TABLE transactions AS
SELECT * FROM (
  SELECT
    customer_id,
    book.book_id AS book_id,
    book.quantity AS quantity
  FROM orders_enriched
) PIVOT (SUM(quantity) FOR book_id IN (
  'B01', 'B02', 'B03', 'B04', 'B05', 'B06',
  'B07', 'B08', 'B09', 'B10', 'B11', 'B12'
  )
);
SELECT * FROM transactions;